In [1]:
# Load .npy files
import numpy as np

# Load features and labels
train_manual    = np.load("featuresets/train_manual_features.npy")
test_manual     = np.load("featuresets/test_manual_features.npy")
train_mobilenet = np.load("featuresets/train_mobilenet_features.npy")
test_mobilenet  = np.load("featuresets/test_mobilenet_features.npy")
train_densenet  = np.load("featuresets/train_densenet_features.npy")
test_densenet   = np.load("featuresets/test_densenet_features.npy")
train_labels    = np.load("featuresets/train_manual_labels.npy")
test_labels     = np.load("featuresets/test_manual_labels.npy")

# Combine features
X_train = np.hstack((train_manual, train_mobilenet, train_densenet))
X_test  = np.hstack((test_manual, test_mobilenet, test_densenet))

print("Manual:", train_manual.shape)
print("MobileNet:", train_mobilenet.shape)
print("DenseNet:", train_densenet.shape)
print("Combined Train:", X_train.shape)
print("Combined Test:", X_test.shape)
print("Labels:", train_labels.shape)

# Remove Class 6
class_to_remove = 6
train_mask = train_labels != class_to_remove
test_mask  = test_labels != class_to_remove

X_train_filtered = X_train[train_mask]
X_test_filtered  = X_test[test_mask]
y_train_filtered = train_labels[train_mask]
y_test_filtered  = test_labels[test_mask]

print("Filtered Train Shape:", X_train_filtered.shape)
print("Filtered Test Shape:", X_test_filtered.shape)

# ⚙️ Import classifiers
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Base learners
base_learners = [
    ('svm', SVC(probability=True, C=10, gamma='scale', random_state=42)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
    ('rf', RandomForestClassifier(n_estimators=150, random_state=42))
]

# Meta-model
meta_model = LogisticRegression(max_iter=1000, random_state=42)

# Build stacking classifier
stacked_clf = StackingClassifier(estimators=base_learners, final_estimator=meta_model, cv=5)

# Train
stacked_clf.fit(X_train_filtered, y_train_filtered)

# Predict
y_pred = stacked_clf.predict(X_test_filtered)

# Evaluate
print("\n🔗 [Stacked Model Results]")
print(classification_report(y_test_filtered, y_pred))
print("Stacked Accuracy:", accuracy_score(y_test_filtered, y_pred))


Manual: (8400, 14)
MobileNet: (8400, 1280)
DenseNet: (8400, 1024)
Combined Train: (8400, 2318)
Combined Test: (1400, 2318)
Labels: (8400,)
Filtered Train Shape: (6900, 2318)
Filtered Test Shape: (1200, 2318)


D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:37:11] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:50:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:52:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-packages\xgboost\training.py:183: UserWarning: [10:53:39] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
D:\Anaconda\Lib\site-pac


🔗 [Stacked Model Results]
              precision    recall  f1-score   support

           0       0.76      0.74      0.75       200
           1       0.90      0.92      0.91       200
           2       0.77      0.77      0.77       200
           3       0.83      0.83      0.83       200
           4       0.90      0.89      0.89       200
           5       0.79      0.81      0.80       200

    accuracy                           0.82      1200
   macro avg       0.82      0.82      0.82      1200
weighted avg       0.82      0.82      0.82      1200

Stacked Accuracy: 0.8241666666666667
